In [1]:
# Submitted By: Rohit Rai (MIT2020098)
# Implementation of Apriori Algorithm

import numpy as np

Loading the data

In [2]:
path_to_data = "transaction_data.txt"
min_support = 2/9
min_conf = 0.3
order = ['I' + str(i) for i in range(1,6)]
order

['I1', 'I2', 'I3', 'I4', 'I5']

In [3]:
def load_transactions(path_to_data, order):
    Transactions = []
    with open(path_to_data, 'r') as fid:
        for lines in fid:
            str_lines = list(lines.strip().split(','))
            _t = list(np.unique(str_lines))
            _t.sort(key = lambda x: order.index(x))
            Transactions.append(_t)
    return Transactions        


Transactions = load_transactions(path_to_data, order)
num_trans = len(Transactions)
Transactions

[['I1', 'I2', 'I5'],
 ['I2', 'I4'],
 ['I2', 'I3'],
 ['I1', 'I2', 'I4'],
 ['I1', 'I3'],
 ['I2', 'I3'],
 ['I1', 'I3'],
 ['I1', 'I2', 'I3', 'I5'],
 ['I1', 'I2', 'I3']]

Initialization

Create C1

In [4]:
C = {}
L = {}
itemset_size = 1
Discarded = {itemset_size : []}
C.update({itemset_size : [ [f] for f in order]})

In [5]:
C

{1: [['I1'], ['I2'], ['I3'], ['I4'], ['I5']]}

In [6]:
# helper function
def print_table(T, supp_count):
  print(" Itemset : Frequency")
  for k in range(len(T)):
    print("{} : {} ".format(T[k], supp_count[k]))
  print("\n\n")  

Create L1

In [8]:
# helper function
def count_occurences(itemset, Transactions):
  count=0
  for i in range(len(Transactions)):
    if set(itemset).issubset(set(Transactions[i])):
      count += 1
  return count    

# helper function
def get_frequent(itemsets, Transactions, min_support, prev_discarded):
  L = []
  supp_count = []
  new_discarded = []

  k = len(prev_discarded.keys())
  for s in range(len(itemsets)):
    discarded_before = False
    if k > 0:
      for it in prev_discarded[k]:
        if set(it).issubset(set(itemsets[s])):
          discarded_before = True
          break
    
    if not discarded_before:
      count = count_occurences(itemsets[s], Transactions)
      if count/len(Transactions) >= min_support:
        L.append(itemsets[s])
        supp_count.append(count)
      else: 
        new_discarded.append(itemsets[s])        

  return L, supp_count, new_discarded  



supp_count_L = {}
f, sup, new_discarded = get_frequent(C[itemset_size], Transactions, min_support, Discarded)
Discarded.update({itemset_size : new_discarded})
L.update({itemset_size : f})
supp_count_L.update({itemset_size : sup})


In [9]:
print("L1: ")
print_table(L[1], supp_count_L[1])

L1: 
 Itemset : Frequency
['I1'] : 6 
['I2'] : 7 
['I3'] : 6 
['I4'] : 2 
['I5'] : 2 





In [10]:
#

C[k] (we need the join step from L[k-1]) and
L[k]


In [11]:
# helper function
def join_two_itemsets(it1, it2, order):
  it1.sort(key = lambda x: order.index(x))
  it2.sort(key = lambda x: order.index(x))
  for i in range(len(it1)-1):
    if it1[i] != it2[i]:
      return []
  if order.index(it1[-1]) < order.index(it2[-1]):
    return it1 + [it2[-1]]    
  return []

# helper function
def join_set_itemsets(set_of_its, order):
  C = []
  for i in range(len(set_of_its)):
    for j in range(i+1, len(set_of_its)):
      it_out = join_two_itemsets(set_of_its[i], set_of_its[j], order)
      if len(it_out) > 0:
        C.append(it_out)
  return C

In [12]:
k = itemset_size + 1
convergence = False
while not convergence:
  C.update({ k : join_set_itemsets(L[k-1], order)})
  print("Table C{}: \n".format(k))
  print_table(C[k], [count_occurences(it, Transactions) for it in C[k]])
  f, sup, new_discarded = get_frequent(C[k], Transactions, min_support, Discarded)
  Discarded.update({ k : new_discarded})
  L.update({ k : f})
  supp_count_L.update({ k : sup })
  if len(L[k]) == 0:
    convergence = True
  else:
    print("Table L{}: \n".format(k))
    print_table(L[k], supp_count_L[k])  
  k += 1


Table C2: 

 Itemset : Frequency
['I1', 'I2'] : 4 
['I1', 'I3'] : 4 
['I1', 'I4'] : 1 
['I1', 'I5'] : 2 
['I2', 'I3'] : 4 
['I2', 'I4'] : 2 
['I2', 'I5'] : 2 
['I3', 'I4'] : 0 
['I3', 'I5'] : 1 
['I4', 'I5'] : 0 



Table L2: 

 Itemset : Frequency
['I1', 'I2'] : 4 
['I1', 'I3'] : 4 
['I1', 'I5'] : 2 
['I2', 'I3'] : 4 
['I2', 'I4'] : 2 
['I2', 'I5'] : 2 



Table C3: 

 Itemset : Frequency
['I1', 'I2', 'I3'] : 2 
['I1', 'I2', 'I5'] : 2 
['I1', 'I3', 'I5'] : 1 
['I2', 'I3', 'I4'] : 0 
['I2', 'I3', 'I5'] : 1 
['I2', 'I4', 'I5'] : 0 



Table L3: 

 Itemset : Frequency
['I1', 'I2', 'I3'] : 2 
['I1', 'I2', 'I5'] : 2 



Table C4: 

 Itemset : Frequency
['I1', 'I2', 'I3', 'I5'] : 1 





Generating Association Rules

In [13]:
from itertools import combinations, chain
def powerset(s):
  return list(chain.from_iterable(combinations(s, r) for r in range(1, len(s) + 1))) 

In [20]:
def write_rules(X, X_S, S, conf, supp, lift, num_transactions):
  out_rules = ""
  out_rules += "Freq. Itemset: {}\n".format(X)
  out_rules += "  Rule: {} -> {} \n".format(list(S), list(X_S))
  out_rules += "  Conf: {0:2.3f} ".format(conf)
  out_rules += "  Supp: {0:2.3f} ".format(supp / num_trans)
  out_rules += "  Lift: {0:2.3f} ".format(lift)
  out_rules += "\n "
  return out_rules


In [21]:
assoc_rules_str = ""

for i in range(1, len(L)):
  for j in range(len(L[i])):
    s = powerset(L[i][j])
    s.pop()
    for z in s:
      S = set(z)
      X = set(L[i][j])
      X_S = set(X-S)
      sup_x = count_occurences(X, Transactions)
      sup_x_s = count_occurences(X_S, Transactions)
      conf = sup_x / count_occurences(S, Transactions)
      lift = conf / ( sup_x_s / num_trans )
      if conf >= min_conf and sup_x >= min_support:
        assoc_rules_str += write_rules(X, X_S, S, conf, sup_x, lift, num_trans)



In [22]:
print(assoc_rules_str)

Freq. Itemset: {'I2', 'I1'}
  Rule: ['I1'] -> ['I2'] 
  Conf: 0.667   Supp: 4.000   Lift: 0.857 
 Freq. Itemset: {'I2', 'I1'}
  Rule: ['I2'] -> ['I1'] 
  Conf: 0.571   Supp: 4.000   Lift: 0.857 
 Freq. Itemset: {'I1', 'I3'}
  Rule: ['I1'] -> ['I3'] 
  Conf: 0.667   Supp: 4.000   Lift: 1.000 
 Freq. Itemset: {'I1', 'I3'}
  Rule: ['I3'] -> ['I1'] 
  Conf: 0.667   Supp: 4.000   Lift: 1.000 
 Freq. Itemset: {'I5', 'I1'}
  Rule: ['I1'] -> ['I5'] 
  Conf: 0.333   Supp: 2.000   Lift: 1.500 
 Freq. Itemset: {'I5', 'I1'}
  Rule: ['I5'] -> ['I1'] 
  Conf: 1.000   Supp: 2.000   Lift: 1.500 
 Freq. Itemset: {'I2', 'I3'}
  Rule: ['I2'] -> ['I3'] 
  Conf: 0.571   Supp: 4.000   Lift: 0.857 
 Freq. Itemset: {'I2', 'I3'}
  Rule: ['I3'] -> ['I2'] 
  Conf: 0.667   Supp: 4.000   Lift: 0.857 
 Freq. Itemset: {'I4', 'I2'}
  Rule: ['I4'] -> ['I2'] 
  Conf: 1.000   Supp: 2.000   Lift: 1.286 
 Freq. Itemset: {'I5', 'I2'}
  Rule: ['I5'] -> ['I2'] 
  Conf: 1.000   Supp: 2.000   Lift: 1.286 
 Freq. Itemset: {'I2'